In [1]:
from google.colab import drive
import sys
drive.mount('/content/drive', force_remount=True)
sys.path.append('/content/drive/MyDrive/attention-model-pl')

Mounted at /content/drive


In [2]:
%ls

drive/  sample_data/


In [3]:
%cd /content/drive/MyDrive/attention-model-pl

/content/drive/MyDrive/attention-model-pl


In [4]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 49.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 65.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.6/774.6 kB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 77.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 47.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.8 MB/s eta 0:00:00


In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
%ls

config.py  lightning_logs/     models/       README.md         tokenizer_en.json  weights/
data/      lightning-train.py  notebooks/    requirements.txt  tokenizer_it.json
LICENSE    logs/               __pycache__/  runs/             utils/


In [7]:
from config import get_config

cfg = get_config()
cfg['batch_size'] = 6
cfg['preload'] = True


from data.data_module import *
from models.lightning_model import *
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateMonitor
import torch

In [8]:
datamodule = BilingualDataModule(cfg)
datamodule.setup()

Generating train split:   0%|          | 0/32332 [00:00<?, ? examples/s]

Max length of source language: 309
Max length of target language: 274


In [9]:
torch.cuda.empty_cache()
pl.seed_everything(1, workers=True)

INFO:lightning_fabric.utilities.seed:Seed set to 1


1

In [10]:
tk_src = datamodule.tokenizer_src  # Assuming datamodule is your instance of BilingualDataModule
tk_tgt = datamodule.tokenizer_tgt

src_vocab_size = tk_src.get_vocab_size()
tgt_vocab_size = tk_tgt.get_vocab_size()

model = LitModel(src_vocab_size, tgt_vocab_size)

In [11]:
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks.progress import TQDMProgressBar

In [12]:
# Tensorboard logger
tb_logger = TensorBoardLogger(save_dir="logs/", name="model")
# Initialize the Lightning Trainer

trainer = pl.Trainer(precision=16,max_epochs=cfg["num_epochs"],logger=tb_logger,
                     accelerator="auto",
        devices=1 if torch.cuda.is_available() else None,  # limiting got iPython runs
        callbacks=[
            ModelCheckpoint(dirpath=cfg['model_folder'], verbose=True),
            LearningRateMonitor(logging_interval="step", log_momentum=True)])

/usr/local/lib/python3.10/dist-packages/lightning_fabric/connector.py:565: `precision=16` is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [13]:
trainer.fit(model, datamodule)

Max length of source language: 309
Max length of target language: 274


/usr/local/lib/python3.10/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:630: Checkpoint directory /content/drive/MyDrive/attention-model-pl/weights exists and is not empty.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name        | Type          | Params
----------------------------------------------
0 | cer_metric  | CharErrorRate | 0     
1 | wer_metric  | WordErrorRate | 0     
2 | bleu_metric | BLEUScore     | 0     
3 | model       | Transformer   | 75.1 M
----------------------------------------------
75.1 M    Trainable params
0         Non-trainable params
75.1 M    Total params
300.532   Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:492: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=1` in the `DataLoader` to improve performance.


--------------------------------------------------------------------------------
    SOURCE: ["Alice took up the fan and gloves, and, as the hall was very hot, she kept fanning herself all the time she went on talking: 'Dear, dear!"]
    TARGET: ['Alice raccolse il ventaglio e i guanti, e perchè la sala sembrava una serra si rinfrescò facendosi vento e parlando fra sè: — Povera me!']
 PREDICTED: ['capirai capirai capirai capirai capirai capirai capirai capirai capirai capirai capirai capirai capirai capirai capirai capirai capirai precipitò precipitò precipitò precipitò precipitò capirai capirai capirai capirai capirai capirai capirai capirai capirai capirai capirai capirai capirai smarrita smarrita smarrita smarrita capirai capirai capirai capirai adatti adatti adatti adatti capirai capirai capirai smarrita smarrita smarrita smarrita smarrita smarrita smarrita smarrita smarrita smarrita smarrita smarrita smarrita smarrita smarrita smarrita smarrita smarrita smarrita smarrita smarrita 

...

Validation: |          | 0/? [00:00<?, ?it/s]

--------------------------------------------------------------------------------
    SOURCE: ["And what is cement?'"]
    TARGET: ['E che cos’è il cemento?']
 PREDICTED: ['E perché cos ’ è mai ?']
--------------------------------------------------------------------------------
    SOURCE: ["And what is cement?'", 'Being a very kind-hearted man he seldom got angry, but when he did, and when his chin twitched, then he was dangerous, as Alexander Vronsky knew.']
    TARGET: ['E che cos’è il cemento?', "Come uomo di cuore, di rado s'arrabbiava, ma quando si arrabbiava e gli tremava il labbro, allora, e Aleksandr Vronskij lo sapeva bene, era pericoloso."]
 PREDICTED: ['E perché cos ’ è mai ?', 'A un uomo molto molto intelligente , ma subito si sentiva sollevata quando egli era sicuro e quando aveva un mento intelligente , come se Vronskij sapeva , come lui , era difficile .']
--------------------------------------------------------------------------------
    SOURCE: ["And what is cement?'"

Validation: |          | 0/? [00:00<?, ?it/s]

--------------------------------------------------------------------------------
    SOURCE: ["'There now! We landlords don't get on well because of the labourers,' said Levin, handing him a tumbler of tea."]
    TARGET: ['— Ed ecco, invece, da noi proprietari tutto va male coi lavoratori — disse Levin, dandogli un bicchiere col tè.']
 PREDICTED: ['— Su , adesso , non c ’ è un ’ abitudine di non — disse Levin , la stizza di Levin .']
--------------------------------------------------------------------------------
    SOURCE: ["'There now! We landlords don't get on well because of the labourers,' said Levin, handing him a tumbler of tea.", 'Before he saw him, he saw his dog.']
    TARGET: ['— Ed ecco, invece, da noi proprietari tutto va male coi lavoratori — disse Levin, dandogli un bicchiere col tè.', 'Prima di vedere Stepan Arkad’ic, ne scorse il cane.']
 PREDICTED: ['— Su , adesso , non c ’ è un ’ abitudine di non — disse Levin , la stizza di Levin .', 'Prima di tutto lo vide , il ca

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.
